In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = [
    'post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
                'rnd_padding', 
                 'aug_padding',
                "zoom_padding"
]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["aug_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/3denses/bio_neurons2/' #'EC_number/archaea/rnn_conv/256rnn/' #3denses/bio_neurons/'
#'EC_number/archaea/stack_conv/10filts_sizeJurtz/'
#1conv/64filts_size5/' #'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 10 #64 #None
kernel_size = [5] #[1,3,5,9,15]#[5] #None
pool_size = 10 #None
n_hid = 256 #None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task2/"
architecture = "only_denses"
#architecture = "conv_dense"
#architecture = "rnn_conv"
#architecture = "stack_conv"

In [3]:
lr=0.0001
optimizer = Adam(lr=lr)

### Loading stuff

In [4]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model - Task 2

In [5]:
model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
#model = building_1convdense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)
#model = building_convrnn_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 
#                                 n_filt, kernel_size, pool_size, n_hid, final_act, folder, optimizer)
#model = building_stackconv_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, 
#                                       kernel_size, pool_size, final_act, folder, optimizer)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 313)         8451      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 76)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 76000)             0         
__________

### Training model

In [ ]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, 
                        val_generator, architecture, max_lenn, dict_size, batch_size, 
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val,
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 1000, 313)         8451      
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000, 76)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 76000)             0         
__________

138/138 [==============================] - 3s 25ms/step - loss: 0.0708 - acc: 0.4634 - val_loss: 0.0166 - val_acc: 0.2050

Epoch 00034: val_acc did not improve from 0.23563
Epoch 35/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0644 - acc: 0.4552 - val_loss: 0.0207 - val_acc: 0.2235

Epoch 00035: val_acc did not improve from 0.23563
Epoch 36/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0693 - acc: 0.4634 - val_loss: 0.0198 - val_acc: 0.2050

Epoch 00036: val_acc did not improve from 0.23563
Epoch 37/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0607 - acc: 0.4509 - val_loss: 0.0167 - val_acc: 0.2139

Epoch 00037: val_acc did not improve from 0.23563
Epoch 38/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0683 - acc: 0.4572 - val_loss: 0.0157 - val_acc: 0.2082

Epoch 00038: val_acc did not improve from 0.23563
Epoch 39/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0635

138/138 [==============================] - 3s 24ms/step - loss: 0.0421 - acc: 0.4716 - val_loss: 0.0139 - val_acc: 0.2031

Epoch 00078: val_acc did not improve from 0.23563
Epoch 79/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0436 - acc: 0.4560 - val_loss: 0.0134 - val_acc: 0.2133

Epoch 00079: val_acc did not improve from 0.23563
Epoch 80/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0429 - acc: 0.4681 - val_loss: 0.0122 - val_acc: 0.2107

Epoch 00080: val_acc did not improve from 0.23563
Epoch 81/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0436 - acc: 0.4640 - val_loss: 0.0087 - val_acc: 0.2050

Epoch 00081: val_acc did not improve from 0.23563
Epoch 82/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0470 - acc: 0.4626 - val_loss: 0.0126 - val_acc: 0.2095

Epoch 00082: val_acc did not improve from 0.23563
Epoch 83/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0401

138/138 [==============================] - 3s 25ms/step - loss: 0.0409 - acc: 0.4669 - val_loss: 0.0077 - val_acc: 0.2056

Epoch 00122: val_acc did not improve from 0.23563
Epoch 123/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0375 - acc: 0.4626 - val_loss: 0.0105 - val_acc: 0.2056

Epoch 00123: val_acc did not improve from 0.23563
Epoch 124/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0380 - acc: 0.4608 - val_loss: 0.0151 - val_acc: 0.2088

Epoch 00124: val_acc did not improve from 0.23563
Epoch 125/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0404 - acc: 0.4685 - val_loss: 0.0068 - val_acc: 0.2050

Epoch 00125: val_acc did not improve from 0.23563
Epoch 126/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0376 - acc: 0.4630 - val_loss: 0.0115 - val_acc: 0.2031

Epoch 00126: val_acc did not improve from 0.23563
Epoch 127/200
138/138 [==============================] - 3s 25ms/step - loss: 0

138/138 [==============================] - 3s 25ms/step - loss: 0.0368 - acc: 0.4651 - val_loss: 0.0084 - val_acc: 0.2043

Epoch 00166: val_acc did not improve from 0.23563
Epoch 167/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0369 - acc: 0.4630 - val_loss: 0.0072 - val_acc: 0.2011

Epoch 00167: val_acc did not improve from 0.23563
Epoch 168/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0377 - acc: 0.4593 - val_loss: 0.0112 - val_acc: 0.2133

Epoch 00168: val_acc did not improve from 0.23563
Epoch 169/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0369 - acc: 0.4685 - val_loss: 0.0113 - val_acc: 0.2050

Epoch 00169: val_acc did not improve from 0.23563
Epoch 170/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0356 - acc: 0.4611 - val_loss: 0.0115 - val_acc: 0.2005

Epoch 00170: val_acc did not improve from 0.23563
Epoch 171/200
138/138 [==============================] - 3s 25ms/step - loss: 0

...

Epoch 1/200
138/138 [==============================] - 5s 39ms/step - loss: 0.8880 - acc: 0.0918 - val_loss: 0.3652 - val_acc: 0.1003

Epoch 00001: val_acc improved from -inf to 0.10026, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task2/ext_padding/1/weights-improvement-001-0.1003.hdf5
Epoch 2/200
138/138 [==============================] - 4s 25ms/step - loss: 0.8337 - acc: 0.1476 - val_loss: 0.3556 - val_acc: 0.1003

Epoch 00002: val_acc did not improve from 0.10026
Epoch 3/200
138/138 [==============================] - 4s 26ms/step - loss: 0.7883 - acc: 0.1794 - val_loss: 0.3476 - val_acc: 0.1009

Epoch 00003: val_acc improved from 0.10026 to 0.10089, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons2/task2/ext_padding/1/weights-improvement-003-0.1009.hdf5
Epoch 4/200
138/138 [==============================] - 3s 25ms/step - loss: 0.7474 - acc: 0.1770 - val_loss: 0.3377 - val_acc: 0.1060



Epoch 00037: val_acc did not improve from 0.22669
Epoch 38/200
138/138 [==============================] - 4s 26ms/step - loss: 0.0963 - acc: 0.4406 - val_loss: 0.0209 - val_acc: 0.2126

Epoch 00038: val_acc did not improve from 0.22669
Epoch 39/200
138/138 [==============================] - 4s 26ms/step - loss: 0.0866 - acc: 0.4463 - val_loss: 0.0220 - val_acc: 0.2133

Epoch 00039: val_acc did not improve from 0.22669
Epoch 40/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0852 - acc: 0.4502 - val_loss: 0.0191 - val_acc: 0.2107

Epoch 00040: val_acc did not improve from 0.22669
Epoch 41/200
138/138 [==============================] - 4s 26ms/step - loss: 0.0834 - acc: 0.4478 - val_loss: 0.0189 - val_acc: 0.2101

Epoch 00041: val_acc did not improve from 0.22669
Epoch 42/200
138/138 [==============================] - 4s 26ms/step - loss: 0.0864 - acc: 0.4493 - val_loss: 0.0194 - val_acc: 0.2095

Epoch 00042: val_acc did not improve from 0.22669
Epoch 43/200
138/13

138/138 [==============================] - 4s 25ms/step - loss: 0.0541 - acc: 0.4603 - val_loss: 0.0091 - val_acc: 0.2056

Epoch 00082: val_acc did not improve from 0.22669
Epoch 83/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0588 - acc: 0.4529 - val_loss: 0.0085 - val_acc: 0.2101

Epoch 00083: val_acc did not improve from 0.22669
Epoch 84/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0533 - acc: 0.4593 - val_loss: 0.0104 - val_acc: 0.2107

Epoch 00084: val_acc did not improve from 0.22669
Epoch 85/200
138/138 [==============================] - 4s 26ms/step - loss: 0.0586 - acc: 0.4592 - val_loss: 0.0135 - val_acc: 0.2107

Epoch 00085: val_acc did not improve from 0.22669
Epoch 86/200
138/138 [==============================] - 4s 25ms/step - loss: 0.0596 - acc: 0.4557 - val_loss: 0.0078 - val_acc: 0.2069

Epoch 00086: val_acc did not improve from 0.22669
Epoch 87/200
138/138 [==============================] - 4s 25ms/step - loss: 0.0541

138/138 [==============================] - 4s 26ms/step - loss: 0.0493 - acc: 0.4614 - val_loss: 0.0089 - val_acc: 0.2043

Epoch 00126: val_acc did not improve from 0.22669
Epoch 127/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0438 - acc: 0.4542 - val_loss: 0.0081 - val_acc: 0.2120

Epoch 00127: val_acc did not improve from 0.22669
Epoch 128/200
138/138 [==============================] - 4s 25ms/step - loss: 0.0455 - acc: 0.4619 - val_loss: 0.0088 - val_acc: 0.2101

Epoch 00128: val_acc did not improve from 0.22669
Epoch 129/200
138/138 [==============================] - 4s 25ms/step - loss: 0.0469 - acc: 0.4592 - val_loss: 0.0113 - val_acc: 0.2095

Epoch 00129: val_acc did not improve from 0.22669
Epoch 130/200
138/138 [==============================] - 4s 25ms/step - loss: 0.0488 - acc: 0.4564 - val_loss: 0.0094 - val_acc: 0.2082

Epoch 00130: val_acc did not improve from 0.22669
Epoch 131/200
138/138 [==============================] - 4s 25ms/step - loss: 0

 23/138 [====>.........................] - ETA: 2s - loss: 0.0412 - acc: 0.2343